In [1]:
# !pip install datasets
# !pip install torchmetrics
# !pip install pytz
# !pip install persiantools
# !pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mount Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# %cd /content/drive/MyDrive

/content/drive/MyDrive


# Configuration

In [1]:
config = {}
config['root_path'] = '/home/user01/'
config['series_ID'] = 102
config['series_desc'] = '''
third week - 29 mehr 1401 - fixed many many things including the targeted backdoor attack, success measures, etc.
'''
config['log_path'] = config['root_path']+'experiments/IMDB/SVM/'
config['log'] = {}

In [2]:
config['poisoning_rate'] = 0.0
# config['num_clean_examples'] = 0
config['learning_rate'] = 0.1
config['batch_size'] = 64
config['num_epochs'] = 500

In [3]:
config['log']['model'] = 'SVM'
config['log']['dataset'] = 'IMDB'
config['log']['task'] = 'binary classification'
config['log']['attack'] = 'no'
config['log']['feature_extraction'] = 'wordcount'
config['log']['attack'] = 'no'
config['log']['pytorch_seed'] = 50
config['log']['numpy_seed'] = 50

In [4]:
config['log_path'] += (str(config['series_ID']) + '-' + config['log']['attack'] + '-' + config['log']['dataset'] + '-' + config['log']['feature_extraction'] + '-' + str(int(config['poisoning_rate']*100)) + '.json').lower().replace(' ', '')

In [5]:
config['log_path']

'/content/drive/MyDrive/experiments/IMDB/SVM/102-no-imdb-wordcount-0.json'

In [6]:
### WE NEED THIS TO IMPORT THE NECESSARY LIBRARIES ###
import sys
sys.path.append(config['root_path'])

In [7]:
import datascience, poisoning, report
from datascience.data import CIFAR10, MNIST, IMDB, BOSTON
from datascience.general import train_dev_test_split, join_np_arrays
from poisoning.process import attacker, defender
from poisoning.eval import attack_success_rate, benign_accuracy, test_accuracy
from report.log import JSONLogger, TextLogger, tehran_datetime
from temporary.functions import _reload
import numpy as np
import torch
from torchmetrics import HingeLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd.functional import hessian, jacobian
from torch.autograd import grad
from torch.nn.utils import _stateless

2022-11-17 19:16:09.673422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 19:16:09.844858: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 19:16:10.537061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: usr/local/cuda-11.8/lib64
2022-11-17 19:16:10.537242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [11]:
# _reload(poisoning.process)
# _reload(poisoning.eval)
# _reload(datascience.data)
# _reload(datascience.general)
# _reload(report.log)

# Loading a Dataset

In [8]:
dataset = IMDB()

HF google storage unreachable. Downloading and preparing it from source


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /home/user01/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
dataset.extract_features('tfidf',1,500,5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
# dataset.separate_examples(config['num_clean_examples'])

In [16]:
# dataset.change_labels({0:-1})

# Prepare the Data

In [17]:
class MyVectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]).to(device), torch.Tensor(self.labels[idx]).to(device)

In [18]:
train_dataset = MyVectorDataset(dataset.x_train, dataset.y_train)
test_dataset = MyVectorDataset(dataset.x_test, dataset.y_test)
# clean_dataset = MyVectorDataset(dataset.x_train_clean, dataset.y_train_clean)

In [19]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
# clean_dataloader = DataLoader(clean_dataset, batch_size=config['batch_size'], shuffle=True)

# Train a Model

In [20]:
class SVM(torch.nn.Module):
    def __init__(self):
        super(SVM, self).__init__() 
        self.linear = torch.nn.Linear(in_features=500, out_features=1, bias=True)
    def forward(self, x):
        output = self.linear(x)
        return output

In [21]:
def output_to_label(out):
    if out >= 0:
      return 1
    else:
      return 0

In [22]:
def make_prediction(model, x_arr):
  outs = list(model(torch.Tensor(x_arr)).squeeze().detach().numpy())
  labels = [output_to_label(out) for out in outs]
  return np.array(labels)

In [23]:
model = SVM()
model = model.to(device)
loss_fn = HingeLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")

In [25]:
def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.cpu().numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.cpu().numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    config['log']['accuracy_'+name] = (100*correct)
    config['log']['loss_'+name] = sum_test_loss
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [26]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch_num)

 Epoch 500 - loss: 0.483779  [19200/25000] 

In [27]:
test_loop(train_dataloader, model, loss_fn, config['num_epochs'], 'Train')

 Epoch 500 - Train Error: Accuracy: 85.0%, Avg loss: 0.005826 

In [28]:
test_loop(test_dataloader, model, loss_fn, config['num_epochs'], 'Test')

 Epoch 500 - Test Error: Accuracy: 83.4%, Avg loss: 0.006214 

In [29]:
y_train_pred = make_prediction(model, dataset.x_train)
y_test_pred = make_prediction(model, dataset.x_test)

In [30]:
config['datetime'] = tehran_datetime()

In [31]:
config

{'series_ID': 102,
 'series_desc': '\nthird week - 29 mehr 1401 - fixed many many things including the targeted backdoor attack, success measures, etc.\n',
 'log_path': '/content/drive/MyDrive/experiments/IMDB/SVM/102-no-imdb-wordcount-0.json',
 'log': {'model': 'SVM',
  'dataset': 'IMDB',
  'task': 'binary classification',
  'attack': 'no',
  'feature_extraction': 'wordcount',
  'pytorch_seed': 50,
  'numpy_seed': 50,
  'accuracy_Train': 84.956,
  'loss_Train': 0.005825586158633232,
  'accuracy_Test': 83.43599999999999,
  'loss_Test': 0.0062140762454271315},
 'poisoning_rate': 0.0,
 'learning_rate': 0.1,
 'batch_size': 64,
 'num_epochs': 500,
 'datetime': '1401-07-30 20:06'}

In [32]:
logger = JSONLogger(config['log_path'], config)
logger.log()